In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
!echo $CUDA_VISIBLE_DEVICES

In [ ]:
!nvidia-smi

In [ ]:
import sys
sys.path.insert(0, "../")

import os
os.chdir("../")

In [ ]:
problems = ["dw4", "lj13", "qm9"]
problem = problems[0]
if problem == "dw4":
    dim = 2
    n_nodes = 4
    from examples.dw4 import *
    import target.double_well as dw4
    target_log_prob_fn = dw4.log_prob_fn
    load_dataset = load_dataset_original
elif problem == "lj13":
    dim = 3
    n_nodes = 13
    from examples.lj13 import *
    import target.leonard_jones as lj13
    target_log_prob_fn = lj13.log_prob_fn
else:
    dim = 3
    n_nodes = 19
    from examples.qm9 import *
    target_log_prob_fn = None

In [ ]:
import yaml
import jax
import matplotlib.pyplot as plt

In [ ]:
USE_64_BIT = False
if USE_64_BIT:
    from jax.config import config
    config.update("jax_enable_x64", True)

In [ ]:
cfg = DictConfig(yaml.safe_load(open(f"examples/config/{problem}.yaml")))
cfg

# Train

In [ ]:
cfg.training.save = False
cfg.logger = DictConfig({"list_logger": None})
cfg.flow.act_norm = False

In [ ]:
# Flow setup
cfg.flow.n_aug = 1
cfg.flow.type = 'along_vector'
cfg.flow.n_layers = 6
cfg.flow.kwargs.n_inner_transforms=3
cfg.flow.nets.egnn.mlp_units = (64, 64)
cfg.flow.nets.mlp_head_config.mlp_units = (64, 64)

In [ ]:
# Training setup.
cfg.training.optimizer.peak_lr = 2e-4
cfg.training.optimizer.init_lr = 1e-4
cfg.training.optimizer.use_schedule = True
cfg.training.optimizer.end_lr = 1e-4
cfg.training.n_epoch = 300
cfg.training.batch_size = 32
cfg.training.optimizer

In [ ]:
experiment_config = create_train_config(cfg, dim=dim, n_nodes=n_nodes,
                                        load_dataset=load_dataset)

logger, state = train(experiment_config)

# 